In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

class CNN(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, 8, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(8, 2, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # self.conv3 = nn.Conv2d(4, 128, kernel_size=3, stride=1, padding=1)
        # self.relu3 = nn.ReLU()
        # self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(128, 64)
        self.relu4 = nn.ReLU()

        self.fc2 = nn.Linear(64, 32)
        self.relu5 = nn.ReLU()

        self.fc3 = nn.Linear(32, 2)

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        # x = self.pool3(self.relu3(self.conv3(x)))
        # print(x.shape)
        x = self.flatten(x)
        # print(x.shape)
        x = self.relu4(self.fc1(x))
        x = self.relu5(self.fc2(x))
        x = self.fc3(x) 

        return x

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import tqdm

In [3]:
data_dir = '/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/HumanvsLLM/HumanvsLLama'

In [4]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize the images
])

In [5]:
train_dataset = torchvision.datasets.ImageFolder(root=data_dir + '/train', transform=transform)
test_dataset = torchvision.datasets.ImageFolder(root=data_dir + '/test', transform=transform)


In [6]:
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Step 3: Define the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN(in_channels=3, num_classes=2).to(device)

In [7]:
print(test_dataset.find_classes(data_dir+'/test'))


(['Human', 'Llama'], {'Human': 0, 'Llama': 1})


In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Step 5: Training Loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')
    
    # Validation
    model.eval()
    correct = 0
    total = 0
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            predicted = torch.argmax(outputs, 1)
            TP += ((predicted == 1) & (labels == 1)).sum().item()
            FP += ((predicted == 1) & (labels == 0)).sum().item()
            TN += ((predicted == 0) & (labels == 0)).sum().item()
            FN += ((predicted == 0) & (labels == 1)).sum().item()
            total += labels.size(0)
            #print(labels, predicted)
            correct += (predicted == labels).sum().item()
    
    val_accuracy = correct / total
    print(f'Validation Accuracy: {val_accuracy:.4f}')
    print(f'TP: {TP}, FP: {FP}, TN: {TN}, FN: {FN}')


Epoch [1/10], Loss: 0.3953
Validation Accuracy: 0.8608
TP: 6390, FP: 1456, TN: 5382, FN: 448
Epoch [2/10], Loss: 0.2758
Validation Accuracy: 0.8849
TP: 5782, FP: 518, TN: 6320, FN: 1056
Epoch [3/10], Loss: 0.2469
Validation Accuracy: 0.8966
TP: 5948, FP: 524, TN: 6314, FN: 890
Epoch [4/10], Loss: 0.2347
Validation Accuracy: 0.8991
TP: 6133, FP: 675, TN: 6163, FN: 705
Epoch [5/10], Loss: 0.2219
Validation Accuracy: 0.9004
TP: 6333, FP: 857, TN: 5981, FN: 505
Epoch [6/10], Loss: 0.2144
Validation Accuracy: 0.9029
TP: 5889, FP: 379, TN: 6459, FN: 949
Epoch [7/10], Loss: 0.2095
Validation Accuracy: 0.9066
TP: 6030, FP: 470, TN: 6368, FN: 808
Epoch [8/10], Loss: 0.2048
Validation Accuracy: 0.8989
TP: 5706, FP: 250, TN: 6588, FN: 1132
Epoch [9/10], Loss: 0.2007
Validation Accuracy: 0.9030
TP: 6233, FP: 721, TN: 6117, FN: 605
Epoch [10/10], Loss: 0.1983
Validation Accuracy: 0.9068
TP: 5889, FP: 326, TN: 6512, FN: 949


In [9]:
import math

MCC = (TP * TN - FP * FN) / math.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))

print(f'Matthews Correlation Coefficient (MCC): {MCC:.4f}')

TPR = TP / (TP + FN)
FPR = FP / (FP + TN)

print(f'True Positive Rate (TPR): {TPR:.4f}')
print(f'False Positive Rate (FPR): {FPR:.4f}')


Matthews Correlation Coefficient (MCC): 0.8169
True Positive Rate (TPR): 0.8612
False Positive Rate (FPR): 0.0477


In [10]:
import torch
import torchvision.models as models

model_path = "/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/model/llama_model.pth"

torch.save(model.state_dict(), model_path)
